# Tutorial 08 - OEM Files and Covariance

In version 0.9.0, ANISE introduces the ability to read CCSDS OEM (Orbit Ephemeris Message) files and interpolate them, including covariance data. This tutorial demonstrates:

1.  **Reading an OEM File:** Loading an OEM file into an `Ephemeris` object.
2.  **Interpolation:** Querying state and covariance at arbitrary times.
3.  **Covariance Frames:** Accessing covariance in different frames (RIC, Inertial, etc.).
4.  **Event Integration:** Combining OEM data with event finding to query covariance at specific event times.

In [ ]:
from pathlib import Path
from anise import Almanac
from anise.astro import Ephemeris, LocalFrame, DataType, Orbit, Frame
from anise.constants import Frames
from anise.time import Epoch, Unit
from anise.analysis import Event, StateSpec, FrameSpec

# Load a base Almanac (for frame definitions)
almanac = Almanac("../../data/pck08.pca")
almanac.describe()

## 1. Reading an OEM File

We can load an OEM file directly into an `Ephemeris` object. This object holds the interpolated data.

In [ ]:
oem_path = "../../data/tests/ccsds/oem/LRO_Nyx.oem"
print(f"Loading OEM from {oem_path}")

ephem = Ephemeris.from_ccsds_oem_file(oem_path)
print("Ephemeris loaded:")
print(ephem)

(start, end) = ephem.domain()
print(f"Domain: {start} to {end}")

## 2. Interpolation and Covariance

We can query the covariance at any time within the domain. ANISE supports transforming the covariance into various local frames:
-   **RIC (Radial, In-track, Cross-track):** Also known as RTN.
-   **Inertial:** The frame in which the data is defined (usually J2000).
-   **VNC (Velocity, Normal, Co-normal):** Useful for maneuver planning.
-   **RCN:** Radial, Cross-track, Normal.

In [ ]:
query_time = start + (end - start) * 0.5
print(f"Querying covariance at {query_time}")

# Covariance in RIC frame
covar_ric = ephem.covar_at(query_time, LocalFrame.RIC, almanac)
print("\nCovariance (RIC):")
print(covar_ric)

# Covariance in Inertial frame
covar_inertial = ephem.covar_at(query_time, LocalFrame.Inertial, almanac)
print("\nCovariance (Inertial):")
print(covar_inertial)

## 3. Combining with Event Finding

A powerful workflow is to find events using the `Almanac` and then inspect the covariance at those specific times.

First, we load the OEM into an `Almanac`. We must provide a NAIF ID for the object in the OEM file (e.g., LRO is -85).

In [ ]:
lro_id = -85
# Load OEM into a new Almanac, assigning it the LRO ID
almanac_oem = Almanac.from_ccsds_oem_file(oem_path, lro_id)
# Load PCK for frame definitions (needed for event finding involving other bodies or frames)
almanac_oem = almanac_oem.load("../../data/pck08.pca")

print("Almanac created from OEM:")
almanac_oem.describe(spk=True)

Now we define an event, for example, finding Apoapsis.

In [ ]:
lro_frame = Frame(lro_id, 1) # Assuming J2000 orientation
moon_frame = Frames.MOON_J2000

lro_state_spec = StateSpec(
    target_frame=FrameSpec.Loaded(lro_frame),
    observer_frame=FrameSpec.Loaded(moon_frame),
    ab_corr=None,
)

apolune = Event.apoapsis()

print(f"Searching for apoapsis events...")
apo_events = almanac_oem.report_events(lro_state_spec, apolune, start, end)
print(f"Found {len(apo_events)} apoapsis events.")

Finally, we iterate through the found events and query the covariance from the original `Ephemeris` object at each event time.

In [ ]:
print("Querying covariance at event times:")
for i, event in enumerate(apo_events):
    event_time = event.orbit.epoch
    
    # Query covariance in RIC frame
    cov = ephem.covar_at(event_time, LocalFrame.RIC, almanac_oem)
    
    print(f"\nEvent {i+1} at {event_time}:")
    print(cov)